# Archive my tweets

- badges: true
- categories: [code, twitter]


I want to extract all the tweets I've ever written and convert them to small markdown files so they show up as "posts" on this website.

Posts are organized by date, in the traditional blogging format. But so are tweets, kind of? They're listed chronologically anyway. Maybe I could make one post for each day, and then have all the tweets listed on that page. Each one could have its tweet ID as a header, thus having an internal link. Tweets can link to the tweets that precede them, and maybe even backlink to tweets that follow.

It's not quite block references, but as a way of keeping my second brain under my ownership it should work. And this way if anyone wants to cancel me they'll have a convenient search box and permalinks for it. Even if my account gets deleted, my bad takes can stay up.

In [1]:
#hide_output
import twint
import nest_asyncio
nest_asyncio.apply()


c = twint.Config()

c.Search = 'from:deepfates (to:deepfates OR exclude:replies)'
tweets = []`
c.Store_object = True
c.Store_object_tweets_list = tweets


twint.run.Search(c)

1429185802731458563 2021-08-21 14:57:03 -0600 <deepfates> YES!! thank you we have a winner   https://t.co/daViFVGsL7
1429184350764118017 2021-08-21 14:51:16 -0600 <deepfates> git is an amazing piece of software. i've been using it for over six years and yet i have no idea how to use it
1429175743641235457 2021-08-21 14:17:04 -0600 <deepfates> thanks but ive got plenty of extensions. i need the twitter search keywords so i can do this programatically
1429167633660850182 2021-08-21 13:44:51 -0600 <deepfates> is there a way to search twitter for my own posts and my own threads at the same time?  "exclude:replies" seems to exclude my replies to myself
1429155773955940358 2021-08-21 12:57:43 -0600 <deepfates> Normies resemble each other in normality; every nerd is extremely niche in their own way
1429152399860637709 2021-08-21 12:44:19 -0600 <deepfates> @paleoludic oh shit i tried IT WORKS   can't believe i din't try printing it before!!
1429150073578037252 2021-08-21 12:35:04 -0600 <deepfa

In [2]:
#better save that list to disk before i mess around too much more
import pickle

with open('../assets/tweets.pkl', 'wb') as f:
    pickle.dump(tweets, f)

In [6]:
len(tweets)

412

In [10]:
t = tweets[5]

In [11]:
t.conversation_id, t.datestamp, t.datetime, t.id, t.likes_count, t.link, t.mentions, t.photos, t.quote_url, t.replies_count, t.reply_to, t.retweet, t.retweet_date, t.retweet_id, t.retweets_count, t.source, t.thumbnail, t.timestamp, t.timezone, t.tweet, t.urls, t.user_id, t.user_id_str, t.user_rt, t.user_rt_id, t.username, t.video

('1429150073578037252',
 '2021-08-21',
 '2021-08-21 12:44:19 MDT',
 1429152399860637709,
 1,
 'https://twitter.com/deepfates/status/1429152399860637709',
 [],
 [],
 '',
 2,
 [{'screen_name': 'paleoludic',
   'name': 'The Neoludic',
   'id': '825775322759102465'}],
 False,
 '',
 '',
 0,
 '',
 '',
 '12:44:19',
 '-0600',
 "@paleoludic oh shit i tried IT WORKS   can't believe i din't try printing it before!!",
 [],
 3315205122,
 '3315205122',
 '',
 '',
 'deepfates',
 0)

In [12]:
import requests
import shutil

In [13]:
def dl_image(url):
    filename = '../images/' + url.split('/')[-1]
    r = requests.get(url, stream = True)
    if r.status_code == 200:
        r.raw.decode_content = True
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)
        return(filename)
    else:
        return(None)
    
def image_template(filename):
    return(f'![image from twitter](/{filename[1:]})\n')

    
def get_tweet(t):
    if t.photos == []:
        img_md = ''
    else:
        img_list = [dl_image(url) for url in t.photos]
        img_md = '\n'.join([image_template(o) for o in img_list])

    return(f'''
#### <a href = "{t.link}">*{t.timestamp}*</a>

<font size="5">{t.tweet}</font>

{img_md}

🗨️ {t.replies_count} ♺ {t.retweets_count} 🤍  {t.likes_count}   

---
    ''')

def get_md(tweets, date):
    tweets_text = ''.join(t for t in tweets)
    return(f'''---
title: deepfates log {date}
layout: post
toc: true
comments: false
search_exclude: false
categories: [tweets]
---

{tweets_text}
            ''')

In [14]:
from IPython.display import Markdown

In [15]:
Markdown(get_tweet(t))


#### <a href = "https://twitter.com/deepfates/status/1429152399860637709">*12:44:19*</a>

<font size="5">@paleoludic oh shit i tried IT WORKS   can't believe i din't try printing it before!!</font>



🗨️ 2 ♺ 0 🤍  1   

---
    

In [16]:
yesterday = '2021-08-20'
y_tweets = [tw for tw in tweets if tw.datestamp == yesterday]
len(y_tweets)

55

In [17]:
y_sorted = sorted(y_tweets, key=lambda x: x.datetime)
# [tweet.tweet for tweet in y_sorted]

Too many replies! Let's limit to just mine for now

In [18]:
y_md = get_md([get_tweet(t) for t in y_sorted if "@" not in t.tweet], yesterday)

In [19]:
y_md

'---\ntitle: deepfates log 2021-08-20\nlayout: post\ntoc: true\ncomments: false\nsearch_exclude: false\ncategories: [tweets]\n---\n\n\n#### <a href = "https://twitter.com/deepfates/status/1428619480108867589">*01:26:41*</a>\n\n<font size="5">one day you\'ll find out there really is no corner of Twitter. but by then it will be too late</font>\n\n\n\n🗨️ 3 ♺ 2 \U0001f90d  41   \n\n---\n    \n#### <a href = "https://twitter.com/deepfates/status/1428622106514833410">*01:37:07*</a>\n\n<font size="5">whoa. how did they know  https://t.co/F1O5sOkkqQ</font>\n\n![image from twitter](/./images/E9N8OvJXMAYIaCl.png)\n\n\n🗨️ 3 ♺ 0 \U0001f90d  16   \n\n---\n    \n#### <a href = "https://twitter.com/deepfates/status/1428738515593056264">*09:19:41*</a>\n\n<font size="5">I realizing that I may never go back and rewrite the goofy novel I wrote in my mid-20s. For some reason I always thought I would fix it up, turn it into something better once I knew more and had more talent. but will I?  maybe I should 

In [20]:
with open(f'../_posts/tweets/{yesterday}-tweets.md', 'w') as f:
    print(y_md, file=f)

Okay, that'll do for now. It prints a chronological page of tweets for each day. Linking, video and oter people's tweets will have to come later.

I'll wrap that behavior in a function and pass it my tweets and a set of dates when i have tweeted.

In [21]:
def write_day_page(day, tweets):
    tweets = [tw for tw in tweets if tw.datestamp == day]
    sorted_tweets = sorted(tweets, key=lambda x: x.datetime)
    md = get_md([get_tweet(t) for t in sorted_tweets], day)
    with open(f'../_posts/tweets/{day}-tweets.md', 'w') as f:
        print(md, file=f)

In [22]:
self_tweets = [t for t in tweets if "@" not in t.tweet]

In [23]:
len(self_tweets)

382

In [24]:
len([tw for tw in self_tweets if tw.datestamp == yesterday])

53

In [25]:
write_day_page(yesterday, self_tweets)

In [26]:
days = set([t.datestamp for t in self_tweets])

In [27]:
from tqdm import tqdm

In [ ]:
for day in tqdm(days):
    write_day_page(day, self_tweets)

 80%|████████  | 8/10 [00:08<00:01,  1.03it/s]

I would also liek to do analysis to see how often I tweet. And make a big list of links. Maybe next time.

For now you can find these secret tweet archives by searching in the [Explore](/explore) page

In [ ]:
days

In [102]:
*Sometimes I have to edit the files printed before, because twint only reaches back so far*

SyntaxError: invalid syntax (<ipython-input-102-3dfcbe7a94c7>, line 1)

In [104]:
from pathlib import Path

In [121]:
for file in tqdm(Path('../_posts/tweets').iterdir()):
    if file.suffix == '.md':
        with open(file, 'r') as f:
            text = f.read()
        cleantext = text.replace('/fastpages/','')
        with open(file, 'w') as f:
            print(cleantext, file=f)


489it [00:00, 9396.94it/s]
